介绍如何将seq2seq模型转换为PyTorch可用的前端混合Torch脚本。

### 1. 混合前端

PyTorch提供了将即时模式的代码增量转换为Torch脚本的机制，
torch脚本是一个在python中的静态可分析和可优化的子集。
torch用它来在Python运行时独立进行深度学习。


Torch中torch.jit模块可以找到将即时Pytorch程序转换为Torch脚本的API。
这个模块有两个核心模式用于将即时模式转换为Torch脚本图形的表示：
跟踪tracing + 脚本化 scripting

torch.jit.trace函数接受一个模块/一个函数和一组示例的输入
然后通过函数或模块运行输入示例，同时跟踪遇到的计算步骤
然后输出一个可以展示跟踪流程的基于图形的函数。

跟踪tracing对于不依赖于数据的控制流的直接的函数和模块非常有用，比如标准的CNN。


### 2.预备环境

导入模块。
如使用自己的模型，需要保证MAX_LENGTH常量设置正确。
ps 这个常量定义了在训练过程中允许的最大的句子长度以及模型能够产生的最大句子长度输出。

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import os
import unicodedata
import numpy as np

device = torch.device("cpu")
MAX_LENGTH = 10 # maximum sentence length

# 默认的词向量
PAD_token = 0 # used for padding short sentence
SOS_token = 1 # Start-of-sentence token
EOS_token = 2 # End-of-sentence token

### 3. 模型概述

使用sequence-to-sequence模型。
这种模型的输入是可变长度序列的情况，输出是可变长度序列，不一定是一对一输入影射。
seq2seq模型由两个RNN组成：encode + decoder

##### （1）编码器Encoder
编码器RNN在输入语句中每次迭代一个标记，比如单词。
每次步骤输出一个“输出”向量 + 一个“隐藏状态”向量

隐藏状态向量在之后则传递到下一个步骤。
同时记录输出向量。

编码器将序列中每个坐标代表的文本转换为高维空间中的一组坐标。
解码器将使用这些作为为给定的任务生成有意义的输出。

##### （2） 解码器Decoder

解码器RNN以逐个令牌的方式生成相应语句。
它使用来自于编码器的文本向量和内部隐藏状态来生成序列中的下一个单词。
它继续生成单词，直到输出表示句子结束的EOS语句。

在解码器中使用专注机制attention mechanism来帮助它在输入的某些部分输出时“保持专注”。
在我们的模型中，实现了“全局关注Global attention”模块，并将其作为解码模型中的子模块。

### 4. 数据处理

在训练之前建立的模型词汇表中的每个单词都影射到一个整数索引。
我们使用Voc对象来包含从单词到索引的映射，以及词汇表中的单词总数。
我们将在运行模型之前加载对象。

此外，为了能够评估，必须提供一个字符串输入的工具。
normalizeString函数将字符串中的所有字符串转换成小写，并删除所有非字母字符。
indexesFromSentence函数接受一个单词的句子，并返回相应的单词索引序列。


In [3]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:'PAD', SOS_token:'SOS', EOS_token:'EOS'}
        self.num_words = 3 # 统计SOS， EOS， PAD
        
    def addSentence(self, sentence):
        for word in sentence.split(''):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    # remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.item():
            if v >= min_count:
                keep_words.append(k)
                
        print('keep_words{}/{} = {:.4f}'.format(
            len(keep_words), 
            len(self.word2index),
            len(keep_words) / len(self.word2index)
        ))
        
        # reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:'PAD', SOS_token:'SOS', EOS_token:'EOS'}
        self.num_words = 3 # 默认统计令牌
        for word in keep_words:
            self.addWord(word)
            
# 小写并删除非字母字符
def normalizaString(s):
    s = s.lower()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

# 使用字符串句子，返回单词索引的句子
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split('')] + [EOS_token]

### 5. 定义编码器

通过torch.nn.GRU模块实现编码器RNN。

接受一批语句（嵌入单词的向量）的输入
它在内部遍历这些句子
每次一个标记，计算隐藏状态。

将这个模块初始化为双向的，相当于拥有两个独立的GRUs
一个按照时间顺序遍历序列
另一个按照相反的顺序遍历序列
最终返回两个GRUs之和。

模型是按照批处理进行训练的
所以在EncoderRNN模型的forward函数需要一个填充的输入批处理


In [4]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        # 初始化GRU input_size hidden_size等参数都设置为hidden_size
        # 因为我们输入的大小是一个有多个特征的词向量 == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                         dropout=(0 if n_layers==1 else dropout),
                         bidirectional=True)
    
    def forward(self, input_seq, input_lengths, hidden=None):
        # 将单词索引转换为向量
        embedded = self.embedding(input_seq)
        # 为RNN模块填充批次序序列
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_length)
        # 正向通过GRU
        outputs, hidden = self.gru(packed, hidden)
        # 打开填充
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # 将双向GRU的输出结果加和
        outputs = outputs[:,:, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        # 返回输出以及最终的隐藏状态
        return outputs, hidden

### 6. 定义解码器的注意力模块

注意力模块Attn，将用作解码器模型的子模块。
取当前解码器RNN输出和整个编码器的输出，并返回关注点“能值”energies。
这个关注能值张量attension energies tensor与编码器输出的大小相同
两者最终相乘，得到一个加权张量

其最大值表示在特定时间步长解码的查询语句最重要的部分。

In [5]:
# Luong的注意力层
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method")
        
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
    
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)
    
    def concat_score(self, hidden, encoder_output):
        energy - self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        # 根据给定的计算方法计算注意力权重/能量
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_erergies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        # 转制max_length和batch_size维度
        attn_energies = attn.energies.t()
        
        # 返回softmax归一化概率分数（增加维度）
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

### 7. 定义解码器

还是使用nn.GRU模块作为解码器RNN。
但是这次使用的是单向的GRU。

In [6]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
        
        # 保持参考
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # 定义层
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers==1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        self.attn = Attn(attn_model, hidden_size)
        
    def forward(self, input_step, last_hidden, encoder_outputs):
        # 这步只运行一次
        # 获取当前输入字对应的向量映射
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 通过单向GRU转发
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 通过当前GRU计算注意力权重
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # 注意力权重乘以编码器输出以获得新的加权和 上下文向量
        context = attn_weights.bmm(encoder_output.transpose(0, 1))
        # 使用Luong公式5来连接加权上下文向量和GRU输出
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Luong公式6 来预测下一个单词
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # 返回输出和最终的隐藏状态
        return output, hidden

In [7]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # 格式化输入句子作为批处理
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # 创建长度张量
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 转置批量的维度以匹配模型的期望
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 使用适当的设备
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用earcher解码句子s
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


# 评估来自用户输入的输入(stdin)
def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 获取输入的句子
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 规范化句子
            input_sentence = normalizeString(input_sentence)
            # 评估句子
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 格式化和打印回复句
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

# 规范化输入句子并调用evaluate()
def evaluateExample(sentence, encoder, decoder, searcher, voc):
    print("> " + sentence)
    # 规范化句子
    input_sentence = normalizeString(sentence)
    # 评估句子
    output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))